In [1]:
import matplotlib.pyplot as plt
# from networkx import minimum_cut, DiGraph
from mip import *

from src.read_instance import MDOVRP

In [2]:
# filepath = "Instancias/ES-n78-m2-Q10138.txt"
filepath = "Instancias/ES-n78-m2-Q10138_with_depot.txt"
# filepath = "Instancias/Vrp-Set-A/A-n80-m2-Q60.vrp"

In [3]:
N, D, V, Q, q, c, vehicles_depot, coord_x, coord_y = MDOVRP(filepath)

Usando a formulação $MDOVRP_{2i− flv}$ de Lalla-Ruiz e Mes (2019)


    Lalla-Ruiz, Eduardo, and Martijn Mes. "Mathematical formulations and improvements for the multi-depot open vehicle routing problem." Optimization Letters 15 (2021): 271-286.

In [4]:
print(D)
print(vehicles_depot)

[76, 77]
[4, 4]


In [5]:
#cria o modelo
model = Model('PRVMD', solver_name = GUROBI)

# Variaveis de decisao
x = [[model.add_var(var_type=BINARY) if i!=j else model.add_var(lb=0, ub=0) for i in V] for j in V]
u = [[model.add_var(var_type=CONTINUOUS) if i!=j else model.add_var(lb=0, ub=0) for i in V] for j in V]
y = [model.add_var(var_type=BINARY) for i in N]
w = [model.add_var(var_type=BINARY) for i in N]

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID
Academic license - for non-commercial use only - registered to thiago.giachetto@aluno.ufop.edu.br


\begin{equation}
\label{eq:9}
    x_{ij} \in \{0, 1\}, \quad \forall i, j \in V
\end{equation}

\begin{equation}
\label{eq:10}
    u_{ij} \geq 0, \quad \forall i, j \in V
\end{equation}

\begin{equation}
\label{eq:10.1}
    w_{j} \in \{0, 1\}, \quad \forall j \in N
\end{equation}

In [6]:
# Funcao objetivo
alfa = 0.001

A = alfa
B = 1 - alfa

model.objective = minimize(A*xsum(x[i][j] * c[i,j] for i in V for j in V if i != j) - B*xsum(q[j]*w[j] for j in N))

In [7]:
def objective(alfa, c, q, x, w):
    A = alfa
    B = 1 - alfa
    
    return minimize(A*xsum(x[i][j] * c[i,j] for i in V for j in V if i != j) - B*xsum(q[j]*w[j] for j in N))

\begin{equation}
\label{eq:1}
 Minimizar \quad A\sum_{i \in V }\sum_{j \in V} c_{ij}x_{ij} - B\sum_{j \in N} q_{j}w_{j}
\end{equation}

In [8]:
# Restricao 3
for j in N:
    model += xsum(x[i][j] for i in V if i != j) == w[j]

\begin{equation}
    \sum_{i \in V, i \neq j}x_{ij} = w_{j}, \quad \forall j \in N
    \tag{2}
\end{equation}

In [9]:
# Restricao 7

for j in N:
    model += (xsum(x[i][j] for i in V if i != j) - xsum(x[j][i] for i in N if i != j)) >= 0

\begin{equation}
    \sum_{i \in V, i \neq j}x_{ij} - \sum_{i \in N, i \neq j}x_{ji} \geq 0, \quad \forall j \in N
    \tag {18}
\end{equation}

In [10]:
# Restricao 8
for i in V:
    for j in V:
        if i != j:
            model += x[i][j] + x[j][i] <= 1

\begin{equation}
    x_{ij} + x_{ji} \leq 1, \quad \forall i, j \in V, i \neq j
    \tag {19}
\end{equation}

In [11]:
# Restricao 9
# Ninguem volta para o depósito
model += xsum(x[j][k] for k in D for j in V) == 0

\begin{equation}
    \sum_{k \in D}\sum_{j \in V}x_{jk} = 0
    \tag {20}
\end{equation}

In [12]:
# Restricao 10 -> se cidade não é atendida, restrição é esquecida
for j in N:
    model += (xsum(u[i][j] for i in V if i != j) - xsum(u[j][i] for i in V if i != j) - q[j]) >= -Q*(1 - w[j])

\begin{equation}
    (\sum_{i \in V, i \neq j}u_{ij} - \sum_{i \in V, i \neq j}u_{ji}) - q_{j} \geq -Q(1 - w_{j}), \quad \forall j \in N
    \tag {21}
\end{equation}

    Wolsey, Laurence A. Integer programming (pp. 11). John Wiley & Sons, 2020.

In [13]:
# restrição 3 -> que liga x e w
for j in N:
    xsum(x[i][j] for i in V if i != j) == w[j]

\begin{equation}
    \sum_{i \in V, i \neq j}x_{ij} = w_{j}, \quad \forall j \in N
    \tag {21}
\end{equation}

In [14]:
#Restricao 11
for i in N:
    for j in N:
        model += (Q - q[i])*x[i][j] >= u[i][j]

\begin{equation}
    (Q - q_i) \cdot x_{ij} \geq u_{ij}, \quad \forall i, j \in N
    \tag {22}
\end{equation}

In [15]:
# Restricao 12
for k in D:
    for j in N:
        model += Q * x[k][j] >= u[k][j]

\begin{equation}
    Q \cdot x_{kj} \geq u_{kj}, \quad \forall k \in D, j \in N
    \tag {23}
\end{equation}

In [16]:
# # Restricao 13
# menor distancia de algum deposito ate a cidade
d = [min([c[j, i] for j in D]) for i in N]
# menor distancia entre cidade i e quaisquer outra cidade
r = [min([c[j, i] for j in N if i != j]) for i in N]
# máxima menor distancia entre duas cidades
M = max(r)

# d[i] >= r[i] <=> é mais longe ir de algum depósito do q de outra cidade => y in {0, 1}
# d[i] < r[i] <=> é mais perto sair de algum depósito do q de qualquer outra cidade => y = 1
for i in N:
    model += d[i] + M * y[i] >= r[i]*w[i]

\begin{equation}
    d_{i} + M y_{i} \geq r_{i}w_{i}, \quad \forall k \in N
    \tag {8}
\end{equation}

In [17]:
# Restricao 14

for i in N:
    k_l = D[np.argmin([c[j, i] for j in D], axis=0)]
    model += x[k_l][i] >= y[i]

\begin{equation}
    x_{k'i} \geq y_{i}, \quad \forall i \in N, k' = argmin(c_{ki})_{k \in D}
    \tag {9}
\end{equation}

In [18]:
# Restricao 4 - quantidade veiculos por deposito

for pos, k in enumerate(D):
    model += xsum(x[k][i] for i in N) <= vehicles_depot[pos]
# for lim_vehicle in vehicles_depot:
#     model += xsum(x[k][i] for k in D for i in N) <= lim_vehicle

In [19]:
# Restricao 5 -> na demanda relacionado ao limite de caminhões

model += xsum(q[j]*w[j] for j in N) <= sum(vehicles_depot)*Q

In [20]:
# Restricao 6 -> de distancia maxima entre duas cidades

for i in V:
    for j in V:
        if i != j:
            model += c[i,j]*x[i][j] <= 180

## All feasible found solutions

In [21]:
dist = lambda x, k: sum([(1 if x[i][j].xi(k)>0.98 else 0) * c[i,j] for i in V for j in V if i != j])
demanda = lambda w, k: sum((q[j] if w[j].xi(k)>0.98 else 0) for j in N)


for k in range(model.num_solutions):
    print(f"{k}: dist: {dist(x, k)} dem: {demanda(w, k)}")

In [22]:
model.sol_pool_size

10

In [ ]:
import numpy as np

steps = 30
solutions_obj = set()

for alfa in np.linspace(0, 1, steps, endpoint=True):
    model.objective = objective(alfa, c, q, x, w)
    num_solutions = 0
    
    for i in range(5):
        model.threads = 2
        status = model.optimize(max_seconds=60)
#         status = OptimizationStatus.FEASIBLE
    
        if status in (OptimizationStatus.INFEASIBLE, OptimizationStatus.ERROR):
            break
            
        if model.num_solutions > num_solutions:
            num_solutions = model.num_solutions
            
            for k in range(model.num_solutions):
                obj_calc = (dist(x,k), demanda(w, k))
                
#                 if obj_calc is not in solutions_obj:
                solutions_obj.add(obj_calc)
                    
        
        if status == OptimizationStatus.OPTIMAL:
            break
        
        print(f"number of solutions: {len(solutions_obj)}")

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 2 physical cores, 4 logical processors, using up to 2 threads
Academic license - for non-commercial use only - registered to thiago.giachetto@aluno.ufop.edu.br
Optimize a model with 18324 rows, 12320 columns and 59808 nonzeros
Model fingerprint: 0xf8963e61
Variable types: 6162 continuous, 6158 integer (6158 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [3e+02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+04]

MIP start from previous solve produced solution with objective -80097 (0.01s)
Loaded MIP start from previous solve with objective -80097

Presolve removed 13870 rows and 6497 columns
Presolve time: 0.14s
Presolved: 4454 rows, 5823 columns, 21752 nonzeros
Variable types: 2832 continuous, 2991 integer (2991 binary)

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.02s

Barrier stati

 15792 14297 -80337.000  161   89 -80097.000 -81104.000  1.26%   160  182s
 16691 14619 -80337.000  584  100 -80097.000 -81104.000  1.26%   159  185s
 17323 15271 -81102.659   72   28 -80097.000 -81104.000  1.26%   157  190s
 17998 15634 -81104.000   67   46 -80097.000 -81104.000  1.26%   156  196s
 18348 16202 -80831.000  209   39 -80097.000 -81104.000  1.26%   156  201s
 19274 16843 -81104.000   52   38 -80097.000 -81104.000  1.26%   155  205s
 19947 17445 -81104.000   50   49 -80097.000 -81104.000  1.26%   156  210s
 20478 17881 -81104.000   64   28 -80097.000 -81104.000  1.26%   159  215s
 21790 19385 -80742.000   70   33 -80097.000 -81104.000  1.26%   159  220s
 22797 20203 -80750.000  152   29 -80097.000 -81104.000  1.26%   158  226s
 23502 20659 -80750.000  185   33 -80097.000 -81104.000  1.26%   158  230s
 24454 21550 -80723.000  261   40 -80097.000 -81104.000  1.26%   157  236s
 25008 21870 -80478.000  153   34 -80097.000 -81104.000  1.26%   158  240s

Cutting planes:
  Lift-a

   928   857 -78233.967   28  147 -77083.991 -78233.978  1.49%  92.0   40s
H  951   833                    -77085.36479 -78233.964  1.49%  95.3   42s
H  955   790                    -77085.86341 -78233.964  1.49%  95.4   42s
H  956   753                    -77085.92197 -78233.964  1.49%  95.6   44s
   958   793 -78232.380   35  134 -77085.922 -78233.964  1.49%  95.6   46s
H  999   793                    -77088.96762 -78233.964  1.49%   100   49s
  1050   886 -78232.582   59  130 -77088.968 -78233.962  1.49%   103   53s
  1177   960 -78233.768   41  127 -77088.968 -78233.883  1.49%   110   58s
  1294   967 -78233.568   67  147 -77088.968 -78233.883  1.49%   121   60s

Cutting planes:
  Gomory: 21
  Lift-and-project: 1
  Cover: 1
  Implied bound: 20
  Clique: 7
  MIR: 60
  StrongCG: 1
  Flow cover: 39
  Zero half: 1
  RLT: 5
  Relax-and-lift: 4
  BQP: 1

Explored 1340 nodes (168347 simplex iterations) in 60.01 seconds (51.16 work units)
Thread count was 2 (of 4 available processors)

Sol

 14856 11461 -78233.169   37  211 -77629.199 -78233.169  0.78%  78.6  250s
 14858 11462 -78233.122   38  202 -77629.199 -78233.138  0.78%  78.6  255s
 14860 11464 -78233.092   38  187 -77629.199 -78233.092  0.78%  78.6  260s
 14864 11466 -78232.990   39  189 -77629.199 -78233.090  0.78%  78.7  266s
 14871 11473 -78233.013   41  182 -77629.199 -78233.090  0.78%  78.8  270s
 14886 11550 -78232.875   44  172 -77629.199 -78233.090  0.78%  79.0  283s
H14919 10965                    -77629.88110 -78233.090  0.78%  79.4  283s
 14958 11180 -78232.572   61  154 -77629.881 -78233.090  0.78%  80.0  300s
H15060 10602                    -77630.65276 -78233.090  0.78%  82.1  300s

Cutting planes:
  Gomory: 12
  Implied bound: 30
  Projected implied bound: 1
  Clique: 6
  MIR: 122
  Flow cover: 90
  Zero half: 9
  RLT: 2
  Relax-and-lift: 8

Explored 15205 nodes (1280426 simplex iterations) in 60.00 seconds (87.27 work units)
Thread count was 2 (of 4 available processors)

Solution count 10: -77630.7

H 3641  2960                    -74679.77628 -75363.881  0.92%   142  116s
  4044  3408 -75267.914  339   72 -74679.776 -75363.881  0.92%   136  120s

Cutting planes:
  Gomory: 14
  Lift-and-project: 42
  Implied bound: 30
  Clique: 6
  MIR: 64
  Flow cover: 50
  Zero half: 12
  RLT: 8
  Relax-and-lift: 9
  BQP: 1

Explored 4579 nodes (570281 simplex iterations) in 60.00 seconds (57.71 work units)
Thread count was 2 (of 4 available processors)

Solution count 5: -74679.8 -74679.7 -74678.3 ... -74670.3

Time limit reached
Best objective -7.467977627586e+04, best bound -7.536388050800e+04, gap 0.9161%
number of solutions: 23
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 2 physical cores, 4 logical processors, using up to 2 threads
Academic license - for non-commercial use only - registered to thiago.giachetto@aluno.ufop.edu.br
Optimize a model with 18324 rows, 12320 columns and 59808 nonzeros
Model fingerprint: 0xcec47ada
Variable types: 6162 continuous, 6158 int

  14  -7.25308062e+04 -7.25378876e+04  3.30e-12 1.78e-15  4.42e-04     0s
  15  -7.25323511e+04 -7.25341561e+04  1.25e-12 3.55e-15  1.13e-04     0s
  16  -7.25325995e+04 -7.25328339e+04  3.61e-12 3.55e-15  1.46e-05     0s
  17  -7.25326709e+04 -7.25327094e+04  7.54e-12 7.11e-15  2.40e-06     0s
  18  -7.25326844e+04 -7.25326912e+04  1.24e-11 1.78e-15  4.22e-07     0s
  19  -7.25326862e+04 -7.25326863e+04  2.59e-11 3.55e-15  5.66e-09     0s

Barrier solved model in 19 iterations and 0.45 seconds (0.24 work units)
Optimal objective -7.25326862e+04


Solved with barrier

Root relaxation: objective -7.253269e+04, 1497 iterations, 0.28 seconds (0.13 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -72532.686    0  141 -71891.820 -72532.686  0.89%     -    0s
     0     0 -72527.203    0  165 -71891.820 -72527.203  0.88%     -    2s
     0     0 -72527.197    0  170

 11204  9617 -72369.618  171   66 -71928.650 -72492.965  0.78%  97.0  184s
H11586  9569                    -71932.89128 -72492.965  0.78%  95.2  184s
 11772  9730 -72490.215   25  165 -71932.891 -72492.952  0.78%  94.9  186s
H11968  9782                    -71936.40728 -72492.855  0.77%  94.9  188s
 12089 10197 -72447.387   88   89 -71936.407 -72492.851  0.77%  94.9  191s
 12683 10464 -72379.766   34  117 -71936.407 -72492.851  0.77%  93.7  199s
 12827 10792 -72361.365   49   99 -71936.407 -72492.851  0.77%  94.1  203s
 13218 10886 -72266.106   98   44 -71936.407 -72492.804  0.77%  93.5  205s
 13438 11093 -72490.271   33  149 -71936.407 -72492.801  0.77%  94.4  211s
 13824 11565 -72462.854   43   91 -71936.407 -72492.790  0.77%  95.0  218s
 14217 11720 -72489.244   47  148 -71936.407 -72492.781  0.77%  94.4  223s
 14376 12107 -71981.229  131   72 -71936.407 -72492.760  0.77%  94.8  227s

Cutting planes:
  Gomory: 13
  Lift-and-project: 76
  Cover: 2
  Implied bound: 33
  Projected impl

     0     0 -69664.149    0  171 -69036.363 -69664.149  0.91%     -    4s
     0     0 -69664.036    0  189 -69036.363 -69664.036  0.91%     -    4s
     0     0 -69664.025    0  186 -69036.363 -69664.025  0.91%     -    4s
     0     0 -69663.731    0  193 -69036.363 -69663.731  0.91%     -    4s
     0     0 -69663.714    0  189 -69036.363 -69663.714  0.91%     -    4s
     0     0 -69663.678    0  192 -69036.363 -69663.678  0.91%     -    4s
     0     0 -69663.678    0  187 -69036.363 -69663.678  0.91%     -    5s
     0     2 -69663.678    0  187 -69036.363 -69663.678  0.91%     -    5s
   185   287 -69647.980   40  157 -69036.363 -69662.126  0.91%   104   10s
H  267   287                    -69049.68041 -69662.126  0.89%  92.0   10s
H  752   724                    -69059.24607 -69662.126  0.87%  71.9   14s
   792   725 -69652.514   39  187 -69059.246 -69662.126  0.87%  76.4   15s
H  794   689                    -69059.26690 -69628.409  0.82%  76.2   19s
   798   692 -69255.382  

 11619  9632 -69592.337   97   87 -69234.597 -69620.442  0.56%   116  246s
 11929 10261 -69584.942  186   76 -69234.597 -69620.442  0.56%   118  252s
 12631 10411 -69556.144  372   89 -69234.597 -69620.442  0.56%   115  255s
 12956 10584 -69411.946  477   75 -69234.597 -69620.420  0.56%   117  263s
H13028 10381                    -69250.10110 -69620.416  0.53%   118  263s
 13068 10528 -69617.196   57  166 -69250.101 -69620.271  0.53%   118  266s
 13223 10749 -69613.711   54  178 -69250.101 -69620.271  0.53%   119  271s
 13444 10996 -69608.693   74  176 -69250.101 -69620.266  0.53%   118  276s
 13732 11067 -69608.627   54  118 -69250.101 -69620.261  0.53%   118  281s
 13812 11354 -69387.132   94  110 -69250.101 -69620.259  0.53%   118  286s
 14140 11797 -69554.208   61  137 -69250.101 -69620.259  0.53%   118  291s

Cutting planes:
  Gomory: 12
  Lift-and-project: 1
  Implied bound: 24
  Clique: 6
  MIR: 350
  StrongCG: 5
  Flow cover: 106
  Zero half: 8
  RLT: 6
  Relax-and-lift: 9

Exp

In [ ]:
first_test = [i for i in solutions]

x_plot = [p[0] for p in first_test]
y_plot = [-p[1] for p in first_test]

plt.scatter(x_plot, y_plot)

plt.show()

In [ ]:
# # Resolve o modelo        
# # model.optimize(max_seconds=7200)
# model.objective = objective(0, c, q, x, w)
# model.optimize(max_seconds=60)
# print("status: {} objective value : {} best possible: {}".format(model.status, model.objective_value, model.objective_bound))

In [ ]:
# model.gap

In [ ]:
# for i in range(5):
#     status = model.optimize(max_seconds=30)
    
#     if status == OptimizationStatus.OPTIMAL:
#         print("found optimal")
#     elif status == OptimizationStatus.FEASIBLE:
#         print("found feasible")
#     elif status == OptimizationStatus.NO_SOLUTION_FOUND:
#         print("run again")

---

## Testando resultados

In [ ]:
# dist percorrida
sum([(1 if x[i][j].x>0.5 else 0) * c[i,j] for i in V for j in V if i != j])

In [ ]:
# demanda total atendida
NUM_VEICULOS = sum(vehicles_depot)
print(f"demanda atendida: {sum(q[j]*w[j].x for j in N)} | limite demanda: {NUM_VEICULOS*Q} | total demanda: {sum(q)}")

In [ ]:
# # --- PLOTA O GRAFICO ---
# plt.figure(figsize=(10, 6))
# plt.scatter(coord_x[0:], coord_y[0:])
# for i in N:
#     plt.annotate(f"{i}", (coord_x[i], coord_y[i]))
# for i in D:
#     plt.plot(coord_x[i], coord_y[i], c = 'r', marker = 's')

# for i in V:
#     for j in V:
#         if x[i][j].x > 0:
#             plt.plot([coord_x[i], coord_x[j]], [coord_y[i], coord_y[j]], c='g', zorder=0)
    
# #escala dos eixos
# plt.yticks([i for i in range(1,80,20)]); 
# plt.xticks([i for i in range(1,80,20)]); 

In [ ]:
# para onde cada veículo vai
for k in D:
    print(f"{k} - ", end='')
    print([i for i, el in enumerate(u[k]) if el.x > 0])

In [ ]:
# caminho dos carros olhando arcos
for k in D:
    print(f"Depot: {k}")
    for i, el in enumerate(x[k]):
        if el.x > 0:
            visitados = [i]
            next_list = [j for j, el in enumerate(x[i]) if el.x > 0]
            while len(next_list) > 0:
                if len(next_list) > 1:
                    print(f"ERROR: {next_list}")
                next_el = next_list[0]
                visitados.append(next_el)
                next_list = [j for j, el in enumerate(x[next_el]) if el.x > 0]
            print(visitados)

In [ ]:
# caminho dos carros olhando demanda
for k in D:
    print(f"Depot: {k}")
    for i, el in enumerate(u[k]):
        if int(el.x) > 0:
            visitados = [(i, int(el.x))]
            next_list = [(j, int(el.x)) for j, el in enumerate(u[i]) if int(el.x) > 0]
            while len(next_list) > 0:
                if len(next_list) > 1:
                    print(f"ERROR: {next_list}")
                next_el, value = next_list[0]
                visitados.append((next_el, value))
                next_list = [(j, int(el.x)) for j, el in enumerate(u[next_el]) if int(el.x) > 0]
            print([(*el, q[el[0]]) for el in visitados])

In [ ]:
print("Cidades não atendidas")

# cidades não atendidas (olhando arestas)
print(f"aresta:  {([j for j in N if sum([x[i][j].x for i in V]) < 0.5])}")

# nao atendidos olhando demanda
print(f"demanda: {[j for j in N if (q[j] - (sum([u[i][j].x for i in V if i != j]) - sum([u[j][i].x for i in N if i != j]))) > 0.05]}")

# nao atendidos olhando w
print(f"w:       {[j for j in N if w[j].x < 0.5]}")

In [ ]:
# nao atendidos olhando w
for j in N:
    if w[j].x < 0.5:
        print(f"{j}-> demanda: {q[j]}")

In [ ]:
# nao atendidos olhando demanda
for j in N:
    enter = (sum([u[i][j].x for i in V if i != j]) - sum([u[j][i].x for i in N if i != j]))
    if (q[j] - enter) > 0.05:
        print(f"{j}-> enter: {enter}  need: {q[j]}")

In [ ]:
# nro de veiculos por depot

for k in D:
    print(f"depot {k}: {sum(x[k][i].x  for i in N)} vehicles")

In [ ]:
# cidades que tem mais de dois arcos ligadas a ela (depositos podem ter mais)
print([j for j in V if len([i for i in V if x[j][i].x > 0.5 or x[i][j].x > 0.5]) > 2])

In [ ]:
print(f"distancia media : {sum([c[el] for el in c])/len(c)}")
print(f"distancia maxima: {max([c[el] for el in c])}")
print(f"distancia minima: {min([c[el] for el in c])}")